In [1]:
from arcface.backbones import *
import datetime
import os
import time
import warnings

import torch
import torch.utils.data
import torchvision
from torchvision.transforms import autoaugment, transforms
from torch import nn

from datasets import IrisVerificationDataset
from backbones import get_model

In [2]:
path = os.path.join("..","models","iresnet18","model_47.pth")
checkpoint = torch.load(path, map_location=torch.device('cpu'))

In [3]:
model = iresnet18(num_features=128, dropout=0.5)
model.load_state_dict(checkpoint["model"])

<All keys matched successfully>

In [6]:
model.eval()
print("done")
#for m in model.modules():
#  if m.__class__.__name__.startswith('Dropout'):
#    m.train()
#o1 = model(idataset[0][0][None,:,:,:])
#o2 = model(idataset[0][0][None,:,:,:])

done


In [7]:
dataset_eval = IrisVerificationDataset('../Datasets/iris_verification_NDCSI2013_01_05', transform=transforms.Compose([
    transforms.Resize(112),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
]))
eval_sampler = torch.utils.data.SequentialSampler(dataset_eval)
data_loader_val = torch.utils.data.DataLoader(
    dataset_eval, 
    batch_size=32, 
    sampler=eval_sampler, 
    num_workers=4, 
    pin_memory=True
)

In [9]:
embedding_dict = {}
for bn, (image, target) in enumerate(data_loader_val):
    print(f"Processed: {bn*32/len(dataset_eval)*100}%\r", end="")
    embedding = model(image)
    for i in range(embedding.shape[0]):
        embedding_dict[target[i].item()] = embedding[i].detach()

KeyboardInterrupt: 

In [11]:
save_point = {"embeddings":embedding_dict}
torch.save(save_point, "iresnet18_embeddings.pth")

